In [ ]:
import numpy as np
import numqi
from tqdm import tqdm
import time
from utils import *

In [ ]:
dimA = 3
dimB = 3
r = 1
d_S = 4
N = 1
kmax = (max(r,2)+1)**(dimA*dimB)-r
ret0 = []
ret1 = []
ret2 = []
threshold = 1e-6
fail_count_gm = 0
fail_count_ppt = 0
kwargs = dict(theta0='uniform', num_repeat=3, tol=1e-12, print_every_round=0)
for _ in tqdm(range(N)):
    matrix_subspace = np.array([numqi.random.rand_haar_state(dimA*dimB) for _ in range(d_S)]).reshape(-1, dimA, dimB)
    basis,basis_orth,space_char = numqi.matrix_space.get_matrix_orthogonal_basis(matrix_subspace, field='complex')
    
    t0 = time.time()
    for k in range(1,kmax+1):
        result = numqi.matrix_space.has_rank_hierarchical_method(matrix_subspace, r+1, k)
        if result:
            break
    ret0.append(time.time()-t0)
    
    t1 = time.time()
    model = numqi.matrix_space.DetectRankModel(basis_orth, space_char, rank=r)
    theta_optim = numqi.optimize.minimize(model,**kwargs)
    ret1.append(time.time()-t1)
    
    if (theta_optim.fun > threshold) != result:
        fail_count_gm += 1
    
    t2 = time.time()
    result_ppt = PPT_GM(basis_orth)
    ret2.append(time.time()-t2)
    
    if (result_ppt > threshold) != result:
        fail_count_ppt += 1
        

print(f'{num_state}-dimensional {dimA}x{dimB} subspace')
print('Hierarchical method:', np.mean(ret0), 's')
print('DetectRankModel:', np.mean(ret1), 's')
print('PPT_GM:', np.mean(ret2), 's')
print(f'fail_count_gm: {fail_count_gm}')
print(f'fail_count_ppt: {fail_count_ppt}')

    

